In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
pd.options.display.max_columns = None

df = pd.read_parquet('csvs/master_dataframe.gzip')
df.game_id.sample().item()

400918880

In [3]:
df.game_id.nunique()

47701

In [8]:
map_dict = {}
for id,sub_df in tqdm(df.groupby('game_id')):

    last_row_df = sub_df.iloc[len(sub_df)-1,:].to_frame().transpose()

    if last_row_df.home_score.item() > last_row_df.away_score.item():

        map_dict[id] = sub_df.home.unique().item()

    else:

        map_dict[id] = sub_df.away.unique().item()
df['winner'] = df.game_id.map(map_dict)

100%|██████████| 47701/47701 [01:27<00:00, 545.10it/s]


In [27]:
def assign_quarters(x):

    if x <= 2400 and x > 1800 :
        return 1
    if x <= 1800 and x > 1200:
        return 2
    if x <= 1200 and x > 600:
        return 3
    return 4

df['quarter'] = df.secs_remaining_absolute.map(assign_quarters)
df.sample(3)

,play_id,half,time_remaining_half,secs_remaining,secs_remaining_absolute,description,home_score,away_score,score_diff,play_length,win_prob,naive_win_prob,home,away,home_time_out_remaining,away_time_out_remaining,home_favored_by,game_id,date,shot_x,shot_y,shot_team,shot_outcome,shooter,assist,three_pt,free_throw,possession_before,possession_after,winner,quarter
7,8,1,19:08,2348,2348,Foul on LaDontae Henton.,1,0,1,0,0.522613,0.522613,Marquette,Providence,4,4,NaN,400504551,2014-01-30,NaN,NaN,None,None,None,None,None,None,None,None,Marquette,1
206,207,2,9:08,548,548,Branford Jones missed Layup.,65,59,6,0,0.801281,0.801281,UC Irvine,Long Beach State,6,4,NaN,400595946,2015-01-08,NaN,NaN,None,None,None,None,None,None,None,None,Long Beach State,4
47,48,1,12:50,1970,1970,Michael Javes Turnover.,20,2,18,16,0.875426,0.875426,Little Rock,St Gregory's,4,4,NaN,323392031,2012-12-04,NaN,NaN,None,None,None,None,None,None,None,None,Little Rock,1


In [29]:
from pprint import pprint

In [37]:
master_dict = {key:[] for key in ('game_id','home_team','away_team','quarter_1_win','quarter_2_win','quarter_3_win','quarter_4_win',
                                  'quarter_differential','game_win')}
test = df[df.game_id == 400918880].copy()

for quarter,sub_df in test.groupby('quarter'):
    target_dict = sub_df.iloc[len(sub_df)-1,:].to_dict()
    
    # From the perspective of the home team
    if target_dict['score_diff'] == 0:
        master_dict[f"quarter_{quarter}_win"] = np.nan

    elif target_dict['score_diff'] > 0:
        master_dict[f"quarter_{quarter}_win"].append(1)
    
    else:
        master_dict[f"quarter_{quarter}_win"].append(0)
    
    if target_dict['winner'] == target_dict['home']:
        master_dict['game_win'].append(1)
    else:
        master_dict['game_win'].append(0)
    
    master_dict['game_id'].append(target_dict['game_id'])
    master_dict['home_team'].append(target_dict['home'])
    master_dict['away_team'].append(target_dict['away'])
master_dict

{'game_id': [400918880, 400918880, 400918880, 400918880],
 'home_team': ['Cal Poly', 'Cal Poly', 'Cal Poly', 'Cal Poly'],
 'away_team': ['UC Davis', 'UC Davis', 'UC Davis', 'UC Davis'],
 'quarter_1_win': [1],
 'quarter_2_win': [1],
 'quarter_3_win': [1],
 'quarter_4_win': [1],
 'quarter_differential': [],
 'game_win': [1, 1, 1, 1]}